In [1]:
#!/usr/bin/env python

from fastai.text import *
from fastai.datasets import *
from pathlib import Path
import pandas as pd
from fastai.metrics import *
from fastai.train import *
from fastai.imports import nn, torch
from fastai.callbacks import *

from fastai import *
from fastai.text import * 

import random
import math
import datetime
from sacred import Experiment

from sacred.observers import MongoObserver
import fastai

import news_utils.fastai
from bpemb import BPEmb

import news_utils.clean.german

In [2]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)
itos = dict(enumerate(bpemb_de.words + ['xxpad']))
voc = Vocab(itos)

In [3]:
bpemb_de.encode('Zeitungskommentare sind eine hervorragende Möglichkeit zum Meinungsaustausch.')

['▁zeitungs',
 'komment',
 'are',
 '▁sind',
 '▁eine',
 '▁hervor',
 'ragende',
 '▁möglichkeit',
 '▁zum',
 '▁meinungs',
 'austausch',
 '.']

In [4]:
res = []

for i in range(0, 14):
    print(i)
    df_a = pd.read_pickle('/mnt/data/group07/johannes/ompc/res_proc7_' + str(i) + '.pkl')
    res.append(df_a)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [5]:
res = pd.concat(res, ignore_index=True)

In [6]:
res

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_Article,depth,rank,res
0,1,-1.0,1,9089,2003-04-23 14:52:41.870,deleted,,,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
1,2,-1.0,1,29367,2003-11-04 16:21:57.850,online,"Newsletter ""DER STANDARD""",Ich bin begeistert von den STANDARD - Newslett...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,2,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
2,3,2.0,1,5095,2004-01-28 12:57:28.240,deleted,Auch begeistert...,... Aber momentan funktioniert das Abmelden od...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
3,4,3.0,1,1682,2004-02-03 20:32:39.123,deleted,Abmeldeprobleme,Es ist ganz einfach nervend!\r\nVor kurzem hab...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",2,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
4,5,-1.0,1,3343,2004-03-02 11:37:44.100,online,,und sie als mitarbeiter sind natuerlich objektiv,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,3,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
5,6,-1.0,1,27945,2004-04-07 16:35:02.893,online,erstklassig...,...ist dieser newsletterservice - und deshalb ...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,4,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
6,7,-1.0,1,2081,2004-11-04 16:29:37.297,deleted,,,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,5,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
7,8,-1.0,1,12862,2004-11-30 09:36:32.447,online,mir fehlt beim Nesletter ...,eine Zusammenfassung aller Kommentare.\r\nDas ...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,6,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
8,9,-1.0,1,50,2005-02-22 12:21:19.113,online,,Wenn ich in meinen Einstellungen oder hier mei...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,7,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
9,10,9.0,1,3462,2005-02-26 11:40:56.287,online,allerdings,"kann das bestätigen, bei mir funktioniert's au...",0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,2,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."


In [7]:
import pandas as pd
import sqlite3

from pathlib import Path

conn = sqlite3.connect("/mnt/data/group07/johannes/ompc/million_post_corpus/corpus.sqlite3")

In [8]:
all_comments_ids = pd.read_sql('select ID_Post from Posts where ID_Post not in (select ID_Post from Annotations_consolidated)', conn)

In [9]:
df_all = all_comments_ids.merge(res)

In [10]:
df_all

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_Article,depth,rank,res
0,1,-1.0,1,9089,2003-04-23 14:52:41.870,deleted,,,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
1,2,-1.0,1,29367,2003-11-04 16:21:57.850,online,"Newsletter ""DER STANDARD""",Ich bin begeistert von den STANDARD - Newslett...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,2,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
2,3,2.0,1,5095,2004-01-28 12:57:28.240,deleted,Auch begeistert...,... Aber momentan funktioniert das Abmelden od...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
3,4,3.0,1,1682,2004-02-03 20:32:39.123,deleted,Abmeldeprobleme,Es ist ganz einfach nervend!\r\nVor kurzem hab...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",2,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
4,5,-1.0,1,3343,2004-03-02 11:37:44.100,online,,und sie als mitarbeiter sind natuerlich objektiv,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,3,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
5,6,-1.0,1,27945,2004-04-07 16:35:02.893,online,erstklassig...,...ist dieser newsletterservice - und deshalb ...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,4,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
6,7,-1.0,1,2081,2004-11-04 16:29:37.297,deleted,,,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,5,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
7,8,-1.0,1,12862,2004-11-30 09:36:32.447,online,mir fehlt beim Nesletter ...,eine Zusammenfassung aller Kommentare.\r\nDas ...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,6,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
8,9,-1.0,1,50,2005-02-22 12:21:19.113,online,,Wenn ich in meinen Einstellungen oder hier mei...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,7,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
9,10,9.0,1,3462,2005-02-26 11:40:56.287,online,allerdings,"kann das bestätigen, bei mir funktioniert's au...",0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,2,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."


In [11]:
df_all[df_all["ID_Article"] == 1475]

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_Article,depth,rank,res
99591,104275,-1.0,1475,19823,2015-07-09 11:35:50.787,online,"""Fisch ist ein Luxusartikel geworden""","Wie kann das sein in einem Land, das am Mittel...",0,0,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",0,1,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99592,104276,-1.0,1475,23206,2015-07-09 11:36:04.333,online,,"""Fisch ist ein Luxusartikel geworden""\r\n\r\nE...",26,2,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",0,2,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99593,104277,-1.0,1475,1018,2015-07-09 11:41:24.900,online,,.. Da hab ich gestern in einer SWR Doku aber e...,9,2,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",0,3,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99594,104278,-1.0,1475,14632,2015-07-09 11:42:06.643,online,,Kein uninteressanter (wenn auch wenig überrasc...,16,0,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",0,4,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99595,104279,104276.0,1475,4416,2015-07-09 11:43:10.807,online,,Haben wohl die Fische schlecht erzogen. Spring...,1,2,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",1,1,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99596,104280,104276.0,1475,19360,2015-07-09 11:43:42.557,online,Den GR-Fischern ist das Dynamit ausgegangen,http://www.griechenland-blog.gr/2012/03/makrel...,2,1,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",1,2,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99597,104281,104276.0,1475,16104,2015-07-09 11:44:38.113,online,,Die Griechen wolllen ja nicht irgendeinen 08/1...,3,14,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",1,3,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99598,104282,104276.0,1475,21315,2015-07-09 11:45:07.607,online,,kennen sie die Preise in Griechenland? Seit de...,4,1,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",1,4,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99599,104283,104276.0,1475,20441,2015-07-09 11:45:09.313,online,,das mittelmeer ist schon lange leer gefischt. ...,30,0,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",1,5,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."
99600,104284,-1.0,1475,16354,2015-07-09 11:48:12.303,online,,"Also, während manche Griechen die Medikamente ...",19,0,Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...,2015-07-09 11:29:11.00,Griechen trotzen der Hackordnung der Märkte,"<div class=""section"" id=""content-main"" itempro...",0,5,"[1, 10889, 24919, 10889, 24929, 1195, 11036, 1..."


In [12]:
df_all.iloc[100000]

ID_Post                                                      104684
ID_Parent_Post                                                   -1
ID_Article                                                     1475
ID_User                                                       11656
CreatedAt                                   2015-07-09 16:04:15.873
Status                                                      deleted
Headline                                                           
Body              eventuell bräuchte man einen psychiatrisch ges...
PositiveVotes                                                     0
NegativeVotes                                                     0
Path              Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...
publishingDate                               2015-07-09 11:29:11.00
Title                   Griechen trotzen der Hackordnung der Märkte
Body_Article      <div class="section" id="content-main" itempro...
depth                                           

In [13]:
df_all.iloc[100000 - 1]

ID_Post                                                      104683
ID_Parent_Post                                               104624
ID_Article                                                     1475
ID_User                                                        1399
CreatedAt                                   2015-07-09 16:00:27.140
Status                                                       online
Headline                      Wie immer: die andern sind Schuld ...
Body              Hätten die Griechen einen ausgeglichenen Staat...
PositiveVotes                                                     4
NegativeVotes                                                     0
Path              Newsroom/Wirtschaft/Wirtschaftpolitik/Finanzma...
publishingDate                               2015-07-09 11:29:11.00
Title                   Griechen trotzen der Hackordnung der Märkte
Body_Article      <div class="section" id="content-main" itempro...
depth                                           

In [15]:
len(df_all.iloc[100000+9]['res'])

734

In [16]:
bpemb_de.decode_ids(df_all.iloc[100000+51]['res'])

'f, babynahrung, hygieneartikel und so weiter. danke, efcharisto und schönen tag noch! xxd xxc ich kann nicht wirklich mitleid empfinden - kinderbeihilfe bis 00 - dorf der blinden http://www.spiegel.de/wirtschaft/soziales/000-blinde-auf-einer-insel-griechische-behoerden-ueberpruefen-sozialbetrug-a-000000.html - pensionen für tote bis in alle ewigkeit diesem land ist nicht zu helfen. und wie man sehen kann, ist das geld sehr wohl bei der bevölkerung angekommen. noch vor 0 jahren (also als schon lange die böse austerität herrschte) hat die mutter der jetzigen parlamentspräsidentin (syriza) kinderbeihilfe für ihr 00-jähriges töchterl bekommen. reaktion ebendieser: so ist halt das system! jeder greift dort zu, wo er nur kann, weil es das system so leicht möglich macht. warum sollten das andere finanzieren?! xxd xxc herrlich so ganz ohne kapitalismus und bargeld. die neue traumdestination aller überzeugten linken. xxd xxc die überschrift könnte auch in der bildzeitung stehen qualutätsjourna

In [17]:
EX_PA = Path('/mnt/data/group07/johannes/ompc/pplmexp_short7')

In [34]:
# df_all['res'] = df_all['res'].apply(lambda x: x[-200:])

In [18]:
bpemb_de.decode_ids(df_all.iloc[100000]['res'])

'hen von: manchen griechen. die haben natürlich mitgeschnitten. und zu glauben, mit kapitalsperren könnte ein varoufakis verhindern, dass zb waffenhändler ihr geld ins ausland transferieren, ist nur naiv.  ⁇ 0 ⁇  http://derstandard.at/0000000000000/militaerbudget-griechenland-die-krise-und-das-militaer xxd xxc eine stimme aus dem griechischen verhandlungsteam: http://www.mediapart.fr/journal/international/000000/we-underestimated-their-power-greek-government-insider-lifts-lid-five-months-humiliation-and-blackm xxd xxc "deutschland hat versagt" "deutschland hat versagt" http://www.euractiv.de/sections/finanzen-und-wirtschaft/schuldenkrise-griechenland-deutschland-hat-kollektiv-versagt-000000 xxd xxc ich bin die weltuntergansartikel über griechenland wirklich leid. nein, es ist nicht wie im krieg. medikamente sind offenbar, wenn auch schwieriger, zu bekommen. man kann 00 euro pro tag abheben, das sollte also durchaus reichen sich mit lebensmittel zu versorgen (ich glaube niemand stirbt a

In [19]:
df_all_train = df_all[df_all['ID_Article'] < 11500]
df_all_val = df_all[df_all['ID_Article'] >= 11500]

data_lm_ft = TextLMDataBunch.from_ids(bs=128, path=EX_PA,vocab=voc, train_ids=df_all_train['res'], valid_ids=df_all_val['res'])

In [21]:
data_lm_ft.save()

In [22]:
out = Path("/mnt/data/group07/johannes/ompc/data_ann_pp_short7")

In [23]:
ann_ids = pd.read_sql('select * from (Annotations_consolidated natural join CrossValSplit)', conn)

In [24]:
df_all

,ID_Post,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_Article,depth,rank,res
0,1,-1.0,1,9089,2003-04-23 14:52:41.870,deleted,,,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
1,2,-1.0,1,29367,2003-11-04 16:21:57.850,online,"Newsletter ""DER STANDARD""",Ich bin begeistert von den STANDARD - Newslett...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,2,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
2,3,2.0,1,5095,2004-01-28 12:57:28.240,deleted,Auch begeistert...,... Aber momentan funktioniert das Abmelden od...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
3,4,3.0,1,1682,2004-02-03 20:32:39.123,deleted,Abmeldeprobleme,Es ist ganz einfach nervend!\r\nVor kurzem hab...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",2,1,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
4,5,-1.0,1,3343,2004-03-02 11:37:44.100,online,,und sie als mitarbeiter sind natuerlich objektiv,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,3,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
5,6,-1.0,1,27945,2004-04-07 16:35:02.893,online,erstklassig...,...ist dieser newsletterservice - und deshalb ...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,4,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
6,7,-1.0,1,2081,2004-11-04 16:29:37.297,deleted,,,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,5,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
7,8,-1.0,1,12862,2004-11-30 09:36:32.447,online,mir fehlt beim Nesletter ...,eine Zusammenfassung aller Kommentare.\r\nDas ...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,6,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
8,9,-1.0,1,50,2005-02-22 12:21:19.113,online,,Wenn ich in meinen Einstellungen oder hier mei...,0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,7,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."
9,10,9.0,1,3462,2005-02-26 11:40:56.287,online,allerdings,"kann das bestätigen, bei mir funktioniert's au...",0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",1,2,"[1, 10889, 24919, 10889, 24929, 10889, 24919, ..."


In [25]:
df_all = ann_ids.merge(res)

In [26]:
df_all

,ID_Post,Category,Value,Fold,ID_Parent_Post,ID_Article,ID_User,CreatedAt,Status,Headline,Body,PositiveVotes,NegativeVotes,Path,publishingDate,Title,Body_Article,depth,rank,res
0,79,PossiblyFeedback,1,1,-1.0,1,12071,2015-06-01 08:58:32.363,online,,"ich kann keinen hinweis finden, wo man sich hi...",0,0,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,47,"[1, 72, 56, 93, 3820, 570, 88, 1917, 46, 4751,..."
1,81,PossiblyFeedback,1,1,-1.0,1,27465,2015-10-03 14:06:06.210,online,,Sehr geehrte Community oder cmb!\r\n\r\nBekomm...,0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,48,"[1, 245, 1477, 49, 1744, 59, 5434, 6565, 39, 2..."
2,132,PossiblyFeedback,1,1,-1.0,2,28073,2015-10-09 11:33:07.227,online,Irgendwie...,kommen manche Posts nicht durch :(,0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",0,23,"[1, 448, 8446, 24940, 22854, 303, 847, 3146, 1..."
3,134,PossiblyFeedback,1,1,-1.0,2,20194,2015-11-04 13:29:34.930,online,,Eine Frage: \r\n\r\nGibt es eine Möglichkeit s...,0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",0,24,"[1, 1492, 234, 79, 1835, 3050, 758, 76, 10855,..."
4,139,PossiblyFeedback,1,1,-1.0,2,14020,2015-12-11 14:51:17.540,online,Registrierung,"Ich habe ein Problem: ich bin registriert, übe...",0,0,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",0,26,"[1, 45, 24940, 5261, 5007, 24961, 3077, 439, 4..."
5,171,PossiblyFeedback,1,1,-1.0,3,6917,2015-06-12 22:13:12.577,online,,"Warum gibt es in der App keinen Wetterbereich,...",0,0,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,13,"[1, 10889, 24919, 10889, 24929, 3077, 1942, 24..."
6,174,PossiblyFeedback,1,1,-1.0,3,19460,2015-09-22 22:25:46.360,online,,"ich kann über die App keine ""ansichtssache"" an...",0,0,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,16,"[1, 24914, 9, 1607, 5235, 24940, 499, 31, 3767..."
7,175,PossiblyFeedback,1,1,-1.0,3,2013,2015-09-24 12:45:03.833,online,,"Ziemlich blöd ist ja auch, dass die APP keine ...",2,0,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,17,"[1, 10289, 24935, 47, 10889, 24922, 10889, 249..."
8,182,PossiblyFeedback,1,1,-1.0,3,21315,2016-01-27 08:01:57.927,online,,Wie komme ich zu meiner postinguebersicht?,2,0,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",0,20,"[1, 61, 3860, 4960, 142, 79, 12476, 43, 15597,..."
9,1477,PossiblyFeedback,1,1,1476.0,7,2232,2016-05-03 14:17:12.207,online,,der link geht nit,0,0,Newsroom/User/Community/Austausch,2014-09-26 14:00:11.00,Keine Scheu vor der Community!,"<div class=""section"" id=""content-main"" itempro...",3,1,"[1, 10889, 24919, 10889, 24929, 4950, 45, 679,..."


In [27]:
df_all['res'] = df_all['res'].apply(lambda x: x[-200:])

In [28]:
for i, x in df_all.groupby(['Category']):
    for fold in range(1, 11):
        print(i)
        out_folder = (out/i/str(fold))
        out_folder.mkdir(parents=True, exist_ok=True)
        train = x.query('Fold!=' + str(fold)).to_pickle(out_folder/"train.pkl")
        test = x.query('Fold==' + str(fold)).to_pickle(out_folder/"test.pkl")
#         print(test.shape)

ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
ArgumentsUsed
Discriminating
Discriminating
Discriminating
Discriminating
Discriminating
Discriminating
Discriminating
Discriminating
Discriminating
Discriminating
Inappropriate
Inappropriate
Inappropriate
Inappropriate
Inappropriate
Inappropriate
Inappropriate
Inappropriate
Inappropriate
Inappropriate
OffTopic
OffTopic
OffTopic
OffTopic
OffTopic
OffTopic
OffTopic
OffTopic
OffTopic
OffTopic
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PersonalStories
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
PossiblyFeedback
SentimentNegative
SentimentNegative
SentimentNegative
SentimentNegative
SentimentNegative
SentimentNegative
SentimentNegative
SentimentNegative
Sentim